# ***Precio Promedio por M2 cuadrados***

In [3]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    #if nombre.startswith("df_") and "alfa_q" in nombre and "puebla" in nombre

    if nombre.startswith("df_") and "puebla" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
    and ("jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_feb_2024_puebla',
 'df_jul_2023_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla',
 'df_sep_2023_puebla']

In [7]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Precio':'precio','precio_m2_contruido':'precio_m2_construido','id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','Metros_total':'m2_total','m_total':'m2_total','Precio_M2_Construido':'precio_m2_construido','m_construido':'m2_construido','Precio_M2_Terreno':'precio_m2_terreno'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [8]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['precio', 'm2_total','m2_construido', 'precio_m2_construido', 'precio_m2_terreno']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

### *DF CON RANGOS DE M2 Y SU PRECIO*

In [9]:
new_dataframes_list = []

# Definir los límites de los rangos
rangos = [0, 100, 150, 200, 250, 300, float('inf')]
labels = ['[0-100]', '[100-150]', '[150-200]', '[200-250]', '[250-300]', '[> 300]']

# Iterar sobre cada DataFrame en la lista original
for df in dataframes_list:
    # Crear una copia del DataFrame
    modified_df = df.copy()
    # Asignar el rango correspondiente a cada valor de 'm2_construido'
    modified_df['rango'] = pd.cut(modified_df['m2_total'], bins=rangos, labels=labels, right=False)
    # Crear un nuevo DataFrame solo con las columnas 'rango' y 'precio_m2_terreno'
    new_df = modified_df[['rango', 'precio_m2_terreno']].copy()
    # Agregar el nuevo DataFrame a la lista
    new_dataframes_list.append(new_df)

# Imprimir los primeros registros de cada nuevo DataFrame
for idx, df in enumerate(new_dataframes_list):
    print(f"Primeros registros de nuevo DataFrame {idx + 1}:")
    print(df.head())
    print()  # Agregar una línea en blanco entre cada DataFrame


Primeros registros de nuevo DataFrame 1:
       rango  precio_m2_terreno
0  [100-150]                  0
1  [100-150]                  0
2  [100-150]                  0
3  [150-200]                  0
4  [150-200]                  0

Primeros registros de nuevo DataFrame 2:
     rango  precio_m2_terreno
0  [> 300]           79225.35
1  [> 300]           15760.44
2  [> 300]           19308.94
3  [> 300]           35410.76
4  [> 300]           23358.59

Primeros registros de nuevo DataFrame 3:
     rango  precio_m2_terreno
0  [0-100]           36060.61
1  [0-100]           39205.88
2  [0-100]           51470.59
3  [0-100]           45945.95
4  [0-100]           45945.95

Primeros registros de nuevo DataFrame 4:
     rango  precio_m2_terreno
0  [0-100]            2397.42
1  [0-100]            2400.00
2  [0-100]            2121.21
3  [0-100]            5957.45
4  [0-100]            4538.46

Primeros registros de nuevo DataFrame 5:
     rango  precio_m2_terreno
0  [0-100]           81250.00

### *Promedio de precio_m2_terreno por rango CADA DF* 

In [10]:
# Iterar sobre cada DataFrame modificado en new_dataframes_list
for idx, df in enumerate(new_dataframes_list):
    print(f"Promedio de precio_m2_terreno por rango en el DataFrame {idx + 1}:")
    # Agrupar el DataFrame por la columna 'rango' y calcular el promedio de 'precio_m2_terreno' para cada grupo
    promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()
    print(promedios_por_rango)
    print()  # Agregar una línea en blanco entre cada resultado

Promedio de precio_m2_terreno por rango en el DataFrame 1:
rango
[0-100]      0.0
[100-150]    0.0
[150-200]    0.0
[200-250]    0.0
[250-300]    0.0
[> 300]      0.0
Name: precio_m2_terreno, dtype: float64

Promedio de precio_m2_terreno por rango en el DataFrame 2:
rango
[0-100]      23015.827475
[100-150]    25041.827041
[150-200]    23269.600000
[200-250]    19862.120062
[250-300]    19267.196944
[> 300]      17330.183374
Name: precio_m2_terreno, dtype: float64

Promedio de precio_m2_terreno por rango en el DataFrame 3:
rango
[0-100]      22608.289205
[100-150]    21880.587242
[150-200]    17889.804581
[200-250]    14280.256158
[250-300]    12448.194211
[> 300]      17891.733683
Name: precio_m2_terreno, dtype: float64

Promedio de precio_m2_terreno por rango en el DataFrame 4:
rango
[0-100]      20646.669647
[100-150]    21017.900655
[150-200]    18913.017583
[200-250]    14057.324777
[250-300]    13603.910427
[> 300]      18177.750848
Name: precio_m2_terreno, dtype: float64

Promed

C:\Users\yoe11\AppData\Local\Temp\ipykernel_15988\2294499642.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()


In [11]:
# Inicializar un diccionario para almacenar los promedios de precio por metro cuadrado para cada rango
promedios_por_rango_dict = {}
# Iterar sobre cada DataFrame modificado en new_dataframes_list
for idx, df in enumerate(new_dataframes_list):
    # Agrupar el DataFrame por la columna 'rango' y calcular el promedio de 'precio_m2_terreno' para cada grupo
    promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()
    # Actualizar el diccionario con los promedios calculados
    promedios_por_rango_dict[f'DataFrame_{idx + 1}'] = promedios_por_rango

# Convertir el diccionario en un DataFrame
rango_precio_m2 = pd.DataFrame(promedios_por_rango_dict)
print("DataFrame 'rango_precio_m2':")
rango_precio_m2

DataFrame 'rango_precio_m2':


C:\Users\yoe11\AppData\Local\Temp\ipykernel_15988\225855980.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()


,DataFrame_1,DataFrame_2,DataFrame_3,DataFrame_4,DataFrame_5
rango,,,,,
[0-100],0.0,23015.827475,22608.289205,20646.669647,27596.106287
[100-150],0.0,25041.827041,21880.587242,21017.900655,24604.704831
[150-200],0.0,23269.600000,17889.804581,18913.017583,20753.396131
[200-250],0.0,19862.120062,14280.256158,14057.324777,19296.732387
[250-300],0.0,19267.196944,12448.194211,13603.910427,20575.395857
[> 300],0.0,17330.183374,17891.733683,18177.750848,15982.658463


In [12]:
# Calcular el promedio general de cada rango en todas las columnas de rango_precio_m2
promedio_general_por_rango = rango_precio_m2.mean(axis=1)
# Crear un nuevo DataFrame 'Precio_mean_M2' con los rangos y sus promedios generales
Precio_mean_M2 = pd.DataFrame({'Rango': promedio_general_por_rango.index, 'Precio_m2_mean': promedio_general_por_rango.values})

print("DataFrame 'Precio_mean_M2':")
Precio_mean_M2

DataFrame 'Precio_mean_M2':


,Rango,Precio_m2_mean
0,[0-100],18773.378523
1,[100-150],18509.003954
2,[150-200],16165.163659
3,[200-250],13499.286677
4,[250-300],13178.939488
5,[> 300],13876.465274


# Gráfica

In [13]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9']
porcentajes = Precio_mean_M2['Precio_m2_mean']

# Convertir la columna 'Precio_m2_mean' a números de punto flotante
#Precio_mean_M2['Precio_m2_mean'] = Precio_mean_M2['Precio_m2_mean'].str.replace('$', '').str.replace(',', '').astype(float)
# Convertir los valores a cadenas con el formato deseado
textos_formateados = [f'${round(valor, 2):,.2f}' for valor in Precio_mean_M2['Precio_m2_mean']]

fig = go.Figure()
# Obtener el valor máximo redondeado al próximo múltiplo de 5000
max_valor = int(Precio_mean_M2['Precio_m2_mean'].max() / 5000 + 1) * 5000

# Agregar los datos como barras
fig.add_trace(go.Bar(
    x=Precio_mean_M2['Rango'],  # Eje x: Modalidades
    y=Precio_mean_M2['Precio_m2_mean'],  # Eje y: Porcentajes
    marker_color=colores,  # Especifica los colores de las barras
    text=textos_formateados,  # Texto que se mostrará en las barras (porcentaje)
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))

# Personalizar el diseño de la gráfica
fig.update_layout(
    #title='Precio promedio por M2 totales.',  # Título de la gráfica
    #xaxis=dict(title='Rango'),  # Título del eje x
    yaxis=dict(
        #title='Precio', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
        tickvals=list(range(0, max_valor + 5000, 5000)),  # Valores de los ticks del eje y
        ticktext=[f'${valor:.2f}' for valor in range(0, max_valor + 5000, 5000)]  # Textos de los ticks del eje y
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    height=500,  # Altura de la gráfica
    margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)

)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_PrecioPromedio_m2', carpeta='assets/graficas')
fig.show()